## Here, we prepare the data:
* do "basic" cleaning: drop empty values, drop duplicates, drop subset duplicates => results in df of size 787895 sentence pairs
* run bicleaner-hardrules on 787895 sized df => results in df of size 574034 sentence pairs
* create traindevtest split from bicleaner cleaned file
* remove the sentence pairs from big df, which are contained in dev or test to avoid training on test/dev. Result is train set for basic cleaning model

In [2]:
import pandas as pd
import numpy as np
import csv
import sys
import os
from pathlib import Path

In [7]:
path_ENRO_bisents = os.path.join(os.pardir, "data/00-raw/EN-RO-bisentences.txt")
#quoting is to correctly include quote in quote strings as one field when reading the file
df=pd.read_csv(path_ENRO_bisents, sep='\t', names=['english', 'romanian'], quoting=csv.QUOTE_NONE)

The raw file (df.shape=(2668811, 2)) contains lots of duplicates. Here are some of them:

In [8]:
df_duplcates_sorted=df.groupby(df.columns.tolist(),as_index=False).size().sort_values(by='size',ascending=[False])[:8]
df_duplcates_sorted

,english,romanian,size
174855,Amendment,Amendamentul,59392
872241,–,–,33074
55777,",",",",31713
596108,Text proposed by the Commission,Textul propus de Comisie,24069
433286,Justification,Justificare,18142
93126,1.,1.,14492
561622,Rule,Articolul,11383
528323,Proposal for a regulation,Propunere de regulament,11349


Removing duplicates and NaN values

In [6]:
print(df.shape)
df=df.drop_duplicates()
print(df.shape)
df=df.dropna()
print(df.shape)

(2668811, 2)
(902307, 2)
(902303, 2)


In [5]:
pd.set_option('display.max_colwidth', None)
df.iloc[591524:591529]

,english,romanian
1970266,This recommendation is one of the key elements of the report.,Această recomandare este unul dintre elementele esenţiale ale raportului.
1970267,"Particular attention should be paid to the dangers of illegal, unreported and unregulated fishing.","Ar trebui acordată o atenţie deosebită pericolelor pescuitului ilegal, nedeclarat şi nereglementat."
1970268,Both EU and non-EU countries in the Black Sea basin should take strict measures to fight IUU fishing.,"Atât ţările membre UE, cât şi ţările terţe din bazinul Mării Negre ar trebui să ia măsuri stricte pentru combaterea pescuitului INN."
1970269,This could only be achieved through the active cooperation of the European Commission and the Black Sea countries.,Acest lucru nu poate fi realizat decât prin cooperarea activă dintre Comisia Europeană şi ţările de la Marea Neagră.
1970270,The Committee on Fisheries has repeatedly stressed the importance of taking into account regional specificities when shaping future policies.,Comisia pentru pescuit a subliniat în repetate rânduri importanţa luării în considerare a specificităţilor regionale la formularea politicilor viitoare.


Found untranslated sentence:

In [6]:
contains_untranslated = df[df['romanian'].str.contains('Pursuant to this differentiation the Conference')]

contains_small_difference = df[df['english'].str.contains('Recovery plan for bluefin tuna in the Eastern')]


Example of freely translated sentence:

In [43]:
contains_freely=train[train.iloc[:, 1].str.contains('joc de cuvinte')]
contains_freely

,english,romanian
71292,"""We """"pass"""" arguments among ourselves, exchange views and experience in order to achieve a common """"goal"""".""","Ne «pasăm» argumentele de la unul la altul, facem schimburi de opinii şi de experienţe pentru a obţine un «obiectiv» [joc de cuvinte în limba engleză, «goal» însemnând în acelaşi timp «obiectiv» şi «gol» n.r.]."


In [7]:
contains_small_difference

,english,romanian
206974,Recovery plan for bluefin tuna in the Eastern Atlantic and Mediterranean,Redresarea stocurilor de ton roşu din Oceanul Atlantic de Est şi din Marea Mediterană
207329,Recovery plan for bluefin tuna in the Eastern Atlantic and the Mediterranean,Redresarea stocurilor de ton roşu din Oceanul Atlantic de Est şi din Marea Mediterană
207509,Recovery plan for bluefin tuna in the Eastern Atlantic and Mediterranean * (debate),Redresarea stocurilor de ton roşu din Oceanul Atlantic de Est şi din Marea Mediterană * (dezbatere)
210136,Recovery plan for bluefin tuna in the Eastern Atlantic and Mediterranean * (vote),Redresarea stocurilor de ton roşu din Oceanul Atlantic de Est şi din Marea Mediterană * (vot)
212386,Recovery plan for bluefin tuna in the Eastern Atlantic and the Mediterranean *,Redresarea stocurilor de ton roşu din Oceanul Atlantic de Est şi din Marea Mediterană *


If we remove also alternative translations we are left with 787895 sentence pairs

In [10]:
df_no_ro_alternative=df.drop_duplicates(subset='english')

see=df_no_ro_alternative[df_no_ro_alternative.duplicated(subset="romanian", keep=False)]
grouped=see.groupby("romanian").size().reset_index(name="counts").sort_values(['counts'], ascending=True)

In [11]:
#examples of romanian duplicate sentences
df.merge(grouped, how="right").iloc[250:259]

,english,romanian,counts
250,2009 would be an extremely difficult year for the citizens and economies of eurozone.,"Recunoscând succesul incontestabil al monedei unice, preşedintele Eurogroup a arătat că adevăratul test de coeziune urmează, 2009 fiind un an extrem de dificil pentru cetăţeni şi economiile din zona euro.",2
251,Collective redress,Recursuri colective,2
252,Seeking justice collectively,Recursuri colective,2
253,Redirecting EU funds to help create jobs for young people,Redirecționarea fondurilor UE pentru crearea de locuri de muncă pentru tineri,2
254,"Plan to redirect regional funds to create jobs could undermine trust, MEPs warn",Redirecționarea fondurilor UE pentru crearea de locuri de muncă pentru tineri,2
255,Recovery plan for bluefin tuna in the Eastern Atlantic and Mediterranean,Redresarea stocurilor de ton roşu din Oceanul Atlantic de Est şi din Marea Mediterană,2
256,Recovery plan for bluefin tuna in the Eastern Atlantic and the Mediterranean,Redresarea stocurilor de ton roşu din Oceanul Atlantic de Est şi din Marea Mediterană,2
257,Citation 1 a (new),Referirea 1a (nouă),2
258,Citation 1a (new),Referirea 1a (nouă),2


In [12]:
#df_no_en_alternative=df.drop_duplicates(subset='romanian')
#df_no_en_alternative.shape

df_no_alternatives=df_no_ro_alternative.drop_duplicates(subset='romanian')
df_no_alternatives.shape

(787895, 2)

In [13]:
df=df_no_alternatives
#keep the index and add flag --scol and --tcol to bicleaner 
df.to_csv(os.path.join(os.pardir,"/data/02-preprocessed/bisentences_787895.txt"), header=False, sep="\t")

## here, we use the bicleaner tool to apply some more rules on the just generated bisentences_787895.txt

run <code> bicleaner-hardrules /path/to/bisentences_787895.txt -s en -t ro --annotated_output --disable_minimal_length --scol 2 --tcol 3 > annotated_bisentences_787895.txt </code> in terminal because it does not work from jupyter, because only build-in python kernel work and bicleaner needs another python

In [38]:
df_annotated=pd.read_csv(os.path.join(os.pardir,"data/02-preprocessed/annotated_bisentences_787895.txt"), sep="\t", names=['english', 'romanian', 'yesno', 'reason'], quoting=3)
df_annotated

,english,romanian,yesno,reason
0,ORAL QUESTION H-0336/07,ÎNTREBARE ORALĂ H-0336/07,1,keep
1,for Question Time at the part-session in May 2007,pentru timpul afectat întrebărilor din perioada de sesiune mai 2007,1,keep
2,pursuant to Rule 109 of the Rules of Procedure,în conformitate cu articolul 109 din Regulamentul de procedură,1,keep
3,by,de,1,keep
4,Roberta Alma Anastase,Roberta Alma Anastase,0,c_identical
...,...,...,...,...
2668799,Calls on the Commission to take into consideration the results of any studies on the effects of this programme on children's development;,"invită Comisia să ia în considerare rezultatele studiilor disponibile, referitoare la efectele acestui program asupra dezvoltării copiilor;",1,keep
2668801,"Instructs its President to forward this declaration, together with the names of the signatories The list of signatories is published in Annex 3 to the Minutes of 15 March 2012 ( P7_PV-PROV(2012)03-15(ANN3) ).","încredințează Președintelui sarcina de a transmite prezenta declarație, însoțită de numele semnatarilor Lista semnatarilor este publicată în anexa 3 la procesul-verbal din 15 martie 2012 ( P7_PV-PROV(2012)03-15(ANN3) ).",1,keep
2668802,", to the Commission and to the Parliaments of the Member States.",", Comisiei și parlamentelor statelor membre.",1,keep
2668803,The Week : 01-01-2004(s),"""""""Săptămâna"""" : 01-01-2004(s)""",0,c_majority_alpha(left)


In [39]:
df_reason=df_annotated.groupby(['yesno', 'reason'])['english'].count().reset_index(name='count') \
                             .sort_values(['count'], ascending=False)

df_reason

,yesno,reason,count
28,1,keep,574034
2,0,c_identical,131687
6,0,c_majority_alpha(left),23505
5,0,c_length or c_length_bytes,14810
1,0,c_different_language,10774
22,0,"c_reliable_long_language(right, targetlang)",7035
25,0,left.istitle() and right.istitle(),6746
8,0,c_no_breadcrumbs1(left),4913
3,0,c_identical_wo_digits,3644
4,0,c_identical_wo_punct,1838


In [40]:
pd.set_option('display.max_colwidth', None)
#replace the string with another reason string to sample examples of matched sentences
samples=df_annotated.loc[df_annotated['reason']=='c_no_literals([\'Re:\'], left)']
samples.sample(n=2)

,english,romanian,yesno,reason
1213208,Re: Content,Re: Content,0,"c_no_literals(['Re:'], left)"
2439866,Re: Financing of the European Microfinance Facility,Subiect: Finanțarea instrumentului european de microfinanțare,0,"c_no_literals(['Re:'], left)"


In [87]:
title_not_available=df_annotated[df_annotated.iloc[:, 1].str.contains('Acest titlu nu este încă disponibil în toate limbile')]
title_not_available

,english,romanian,yesno,reason
56894,- Report on gender mainstreaming in the work of the committees ( 2005/2149(INI) ) - FEMM Committee - Rapporteur: Anna Záborská ( A6-0478/2006 ),- Acest titlu nu este încă disponibil în toate limbile: Report on gender mainstreaming in the work of the committees ( 2005/2149(INI) ) - FEMM - Raportoare: Anna Záborská ( A6-0478/2006 ),0,c_different_language
56896,- Report on European Road Safety Action Programme - mid-term review ( 2006/2112(INI) ) - TRAN Committee - Rapporteur: Ewa Hedkvist Petersen ( A6-0449/2006 ),- Acest titlu nu este încă disponibil în toate limbile: Report on European Road Safety Action Programme - mid-term review ( 2006/2112(INI) ) - TRAN - Raportoare: Ewa Hedkvist Petersen ( A6-0449/2006 ),0,c_different_language
56897,- Report on the Council's Seventh and Eighth Annual Reports according to Operative Provision 8 of the European Union Code of Conduct on Arms Exports ( 2006/2068(INI) ) - AFET Committee - Rapporteur: Raül Romeva i Rueda ( A6-0439/2006 ),- Acest titlu nu este încă disponibil în toate limbile: Report on the Council's Seventh and Eighth Annual Reports according to Operative Provision 8 of the European Union Code of Conduct on Arms Exports ( 2006/2068(INI) ) - AFET - Raportor: Raül Romeva i Rueda ( A6-0439/2006 ),0,c_different_language
56898,- Report with recommendations to the Commission on the European private company statute ( 2006/2013(INI) ) - JURI Committee - Rapporteur: Klaus-Heiner Lehne ( A6-0434/2006 ),- Acest titlu nu este încă disponibil în toate limbile: Report with recommendations to the Commission on the European private company statute ( 2006/2013(INI) ) - JURI - Raportor: Klaus-Heiner Lehne ( A6-0434/2006 ),0,c_different_language
56906,"- Michał Tomasz Kamiński, on behalf of the UEN Group.","- Acest titlu nu este încă disponibil în toate limbile: Michał Tomasz Kamiński, în numele grupului UEN.",1,keep
...,...,...,...,...
1105420,- Quality management for European Statistics ( 2011/2289(INI) ),- Acest titlu nu este încă disponibil în toate limbile Quality management for European Statistics ( 2011/2289(INI) ),1,keep
1105425,- Women in Political Decision Making - quality and equality ( 2011/2295(INI) ),- Acest titlu nu este încă disponibil în toate limbile Women in Political Decision Making - quality and equality ( 2011/2295(INI) ),1,keep
1105432,- The small scale and artisanal fisheries and the CFP reform ( 2011/2292(INI) ) (opinion: DEVE),- Acest titlu nu este încă disponibil în toate limbile The small scale and artisanal fisheries and the CFP reform ( 2011/2292(INI) ) (aviz: DEVE),0,c_different_language
1105433,- Reporting obligations under Regulation (EC) No 2371/2002 on the conservation and sustainable exploitation of fisheries resources under the CFP ( 2011/2291(INI) ) (opinion: ENVI),- Acest titlu nu este încă disponibil în toate limbile Reporting obligations under Regulation (EC) No 2371/2002 on the conservation and sustainable exploitation of fisheries resources under the CFP ( 2011/2291(INI) ) (aviz: ENVI),0,c_different_language


In [41]:
df_keep=df_annotated.loc[df_annotated['reason']=='keep']
df_keep

,english,romanian,yesno,reason
0,ORAL QUESTION H-0336/07,ÎNTREBARE ORALĂ H-0336/07,1,keep
1,for Question Time at the part-session in May 2007,pentru timpul afectat întrebărilor din perioada de sesiune mai 2007,1,keep
2,pursuant to Rule 109 of the Rules of Procedure,în conformitate cu articolul 109 din Regulamentul de procedură,1,keep
3,by,de,1,keep
5,to the Council,Consiliului,1,keep
...,...,...,...,...
2668795,Calls on the Commission and the Member States to encourage the introduction of the programme ‘Chess in School’ in the educational systems of the Member States;,invită Comisia și statele membre să încurajeze introducerea programului „șahul în instituțiile de învățământ” în sistemele educaționale din statele membre;,1,keep
2668797,"Calls on the Commission, in its forthcoming communication on sport, to pay the necessary attention to the program ‘Chess in School’ and to ensure sufficient funding for it from 2012 onwards;","invită Comisia ca, în viitoarea sa comunicare privind sportul, să acorde atenția cuvenită programului „șahul în instituțiile de învățământ” și să asigure finanțarea suficientă a acestuia începând din 2012;",1,keep
2668799,Calls on the Commission to take into consideration the results of any studies on the effects of this programme on children's development;,"invită Comisia să ia în considerare rezultatele studiilor disponibile, referitoare la efectele acestui program asupra dezvoltării copiilor;",1,keep
2668801,"Instructs its President to forward this declaration, together with the names of the signatories The list of signatories is published in Annex 3 to the Minutes of 15 March 2012 ( P7_PV-PROV(2012)03-15(ANN3) ).","încredințează Președintelui sarcina de a transmite prezenta declarație, însoțită de numele semnatarilor Lista semnatarilor este publicată în anexa 3 la procesul-verbal din 15 martie 2012 ( P7_PV-PROV(2012)03-15(ANN3) ).",1,keep


In [42]:
df_keep=df_keep[["english", "romanian"]]
df_keep

,english,romanian
0,ORAL QUESTION H-0336/07,ÎNTREBARE ORALĂ H-0336/07
1,for Question Time at the part-session in May 2007,pentru timpul afectat întrebărilor din perioada de sesiune mai 2007
2,pursuant to Rule 109 of the Rules of Procedure,în conformitate cu articolul 109 din Regulamentul de procedură
3,by,de
5,to the Council,Consiliului
...,...,...
2668795,Calls on the Commission and the Member States to encourage the introduction of the programme ‘Chess in School’ in the educational systems of the Member States;,invită Comisia și statele membre să încurajeze introducerea programului „șahul în instituțiile de învățământ” în sistemele educaționale din statele membre;
2668797,"Calls on the Commission, in its forthcoming communication on sport, to pay the necessary attention to the program ‘Chess in School’ and to ensure sufficient funding for it from 2012 onwards;","invită Comisia ca, în viitoarea sa comunicare privind sportul, să acorde atenția cuvenită programului „șahul în instituțiile de învățământ” și să asigure finanțarea suficientă a acestuia începând din 2012;"
2668799,Calls on the Commission to take into consideration the results of any studies on the effects of this programme on children's development;,"invită Comisia să ia în considerare rezultatele studiilor disponibile, referitoare la efectele acestui program asupra dezvoltării copiilor;"
2668801,"Instructs its President to forward this declaration, together with the names of the signatories The list of signatories is published in Annex 3 to the Minutes of 15 March 2012 ( P7_PV-PROV(2012)03-15(ANN3) ).","încredințează Președintelui sarcina de a transmite prezenta declarație, însoțită de numele semnatarilor Lista semnatarilor este publicată în anexa 3 la procesul-verbal din 15 martie 2012 ( P7_PV-PROV(2012)03-15(ANN3) )."


## Create a dev and a test set to use for both models

In [44]:
#dev and test sets are created from the bicleaner cleaned set
test=df_keep.sample(n=2000, random_state=42)
temp_train=df_keep.drop(test.index)

dev=temp_train.sample(n=2000, random_state=42)
train=temp_train.drop(dev.index)

In [45]:
#checking some sample sentence pairs from dev and test
checking=test.sample(n=10, random_state=42)
checking

,english,romanian
2013712,"In a related field, DG TAXUD must be adequately staffed to be able to maintain a high level of ambition in the crucial fight against tax fraud and evasion.","În aceeași ordine de idei, DG TAXUD trebuie să dispună de personal suficient pentru a putea menține un nivel ridicat de motivație în lupta crucială împotriva fraudei și a evaziunii."
1668135,"Member States shall not provide incentives with regard to tyres below class C with respect to either fuel efficiency level or wet grip within the meaning of Annex I , Part s A and B respectively .",Statele membre nu oferă stimulente cu privire la pneurile aflate sub nivelul de eficiență a consumului de combustibil sau sub nivelul de aderență la teren umed specifice clasei C în sensul anexei I partea A și respectiv partea B .
1779798,"- the model of the EU logo,",- modelul pentru sigla UE;
2643878,"Stresses that regional security is in the interests both of the EU and of the other actors in the area, namely the Russian Federation, China and the United States, which are all concerned about growing instability and radicalisation in the region as well as porous frontiers with Afghanistan and the resulting drug-trafficking;","subliniază că securitatea regională este atât în interesul UE, cât și al altor actori din zonă, și anume, Federația Rusă, China și SUA, care sunt toate preocupate de creșterea instabilității și a radicalizării în regiune, precum și de frontierele nesigure cu Afganistanul și de traficul de droguri aferent;"
2122792,"What measures does the Commission intend to take to prevent discrimination against Romanians and ensure compliance by the Italian Government with Directive 2004/38/EC OJ L 158, 30.4.2004, p.","Ce măsuri are în vedere Comisia pentru împiedicarea discriminării românilor și respectarea normelor Directivei 2004/38/CE JO L 158, 30.4.2004, p."
1741063,"Since February 2008 she has been a Member of the CEAD Group (Coordination, Communication, Evaluation, Assurance and Development).","Din februarie 2008, este membră a grupului CEAD (coordonare, comunicare, evaluare, asigurare şi dezvoltare)."
60065,Council and Commission statements:,Declaraţii ale Consiliului şi ale Comisiei:
1901989,"(i) where applicable, the existence and the conditions of deposits or other financial guarantees to be paid or provided by the consumer at the request of the trader;","(i) după caz, existența și condițiile aferente avansurilor sau altor garanții financiare care trebuie să fie plătite sau oferite de consumator la cererea comerciantului ;"
1908304,Section 2Powers of ESMA,Secțiunea 2Competențele AEVMP
2548431,"Welcomes the Agency's initiative to revise its budgetary procedures and associated financial circuits, to designate authorising officers and invest in staff training on financial management;","salută inițiativa Agenției de a-și revizui procedurile bugetare și circuitele financiare asociate, de a desemna ordonatori de credite și de a investi în formarea personalului în gestiunea financiară;"


In [20]:
#remove those entries from the bigger df which are now contained in dev or test
basic_train=df.drop(test.index)
basic_cleaning_train=basic_train.drop(dev.index)
basic_cleaning_train.shape

## Too strict!

* By applying all possible rules from the bicleaner tool, filtering was too strict. Disable some rules in bicleaner_hardrules.py and run bicleaner again. 
* The output was bisentences_787895_less_rules.txt which contained 614480 keep sent pairs

(remark: we disabled some rules instead of just selecting from df, because some sentences matched more than one rule)


In [5]:
#here very long sentences are included >=1024 characters long
df_ann_less_rules=pd.read_csv(os.path.join(os.pardir,"/home/bernadeta/BA_code/annotated_bisentences_787895_less_rules.txt"), sep="\t", names=['english', 'romanian', 'yesno', 'reason'], quoting=3)
df_ann_less_keep=df_ann_less_rules.loc[df_ann_less_rules['reason']=='keep']
df_ann_less_keep


,english,romanian,yesno,reason
0,ORAL QUESTION H-0336/07,ÎNTREBARE ORALĂ H-0336/07,1,keep
1,for Question Time at the part-session in May 2007,pentru timpul afectat întrebărilor din perioad...,1,keep
2,pursuant to Rule 109 of the Rules of Procedure,în conformitate cu articolul 109 din Regulamen...,1,keep
3,by,de,1,keep
5,to the Council,Consiliului,1,keep
...,...,...,...,...
2668799,Calls on the Commission to take into considera...,invită Comisia să ia în considerare rezultatel...,1,keep
2668801,Instructs its President to forward this declar...,încredințează Președintelui sarcina de a trans...,1,keep
2668802,", to the Commission and to the Parliaments of ...",", Comisiei și parlamentelor statelor membre.",1,keep
2668803,The Week : 01-01-2004(s),"""""""Săptămâna"""" : 01-01-2004(s)""",1,keep


In [6]:
df_less_rules_reason=df_ann_less_rules.groupby(['yesno', 'reason'])['english'].count().reset_index(name='count') \
                             .sort_values(['count'], ascending=False)

df_less_rules_reason

,yesno,reason,count
16,1,keep,614480
2,0,c_identical,131821
4,0,c_length or c_length_bytes,14859
1,0,c_different_language,11074
13,0,"c_reliable_long_language(right, targetlang)",7744
3,0,c_identical_wo_digits,3644
12,0,"c_reliable_long_language(left, sourcelang)",1485
10,0,c_no_urls(left),1478
5,0,c_no_glued_words(left),385
8,0,c_no_space_noise(left),314


In [37]:
df_ann_less_keep=df_ann_less_keep[["english", "romanian"]]
df_ann_less_keep.sample(n=10)

,english,romanian
89415,The Transport Committee is tabling two reports designed to raise standards of maritime safety .,Comisia pentru Transport va prezenta două rapoarte privind îmbunătăţirea standardelor de siguranţă în transportul maritim.
2443371,"Welcomes affirmations by the Commission that any ACTA agreement will be limited to the enforcement of existing IPRs, with no prejudice for the development of substantive IP law in the European Union;","salută declarațiile Comisiei conform cărora orice acord ACTA se va limita la aplicarea DPI existente, fără a prejudicia elaborarea normelor de drept material în domeniul proprietății intelectuale în Uniunea Europeană;"
494471,Adopted by EP: 23.3.2006,Adoptarea de către PE: 2 aprilie 2009
2310439,rules for ensuring compliance with the obligations;,normele pentru garantarea respectării obligaţiilor;
2173782,exercising its authority to conduct supplementary surveys to ensure that ships entitled to fly its flag in fact comply with mandatory IMO instruments;,exercitarea puterii sale de a derula controale suplimentare pentru a se asigura că navele care au dreptul să îi arboreze pavilionul respectă instrumentele obligatorii ale OMI;
1609862,It would be logical for the use of methyl bromide also to be prohibited in the context of this Regulation.,Ar fi logic ca utilizarea bromurii de metil să fie de asemenea interzisă în contextul prezentului regulament.
1680540,The technical resources count for the accomplishment of effective nuclear safety.,Resursele tehnice sunt importante pentru obținerea unei securități nucleare reale.
2138288,"A technical dossier contains all the necessary data, including toxicological studies, which must be provided by the notifier, who has the burden of proof to demonstrate acceptable use of its substance.","Un dosar tehnic conține toate datele necesare, inclusiv studiile toxicologice, care trebuie să fie furnizate de către solicitant, căruia îi revine sarcina de a demonstra că utilizarea substanței sale este acceptabilă."
1984247,"B) Whereas the Austrian authorities argue that the redundancies were caused by globalisation and major structural changes in the production of mobile phones and devices which was relocated to Asia, in particular China, and in consequence, the production of printed circuit boards (PCBs) followed the same trend (proximity factor);","B) întrucât autoritățile austriece afirmă că aceste concedieri sunt rezultatul globalizării și al modificărilor structurale majore în producția de telefoane și dispozitive mobile, delocalizată în Asia, mai ales în China, și, prin urmare, producția de plăci cu circuite imprimate a urmat aceeași tendință (factor de proximitate);"
2126188,All EU financed projects comply with the eligibility rules of the Instrument for Pre-accession Assistance (IPA) Regulation and related programming documents.,Toate proiectele finanțate de UE sunt conforme cu normele de eligibilitate ale Regulamentului privind Instrumentul de asistență pentru preaderare (IPA) și ale documentelor de programare aferente.


In [55]:
#remove dev and test entries from df_ann_less_keep
train_less_notest=df_ann_less_keep.drop(test.index)
train_less_nodev=train_less_notest.drop(dev.index)
train_less_nodev.shape

(610480, 2)

## Write all df's to files

In [33]:
#write bicleaner cleaned df's to files 
source_train=train.iloc[:,0]
target_train=train.iloc[:,1]
#size 570034
source_train.to_csv (os.path.join(os.pardir,"/data/02-preprocessed/bicleaner_cleaned/bicleaner_source_train.txt") , index = None, header = False)
target_train.to_csv (os.path.join(os.pardir,"/data/02-preprocessed/bicleaner_cleaned/bicleaner_target_train.txt") , index = None, header = False)

source_test=test.iloc[:,0]
target_test=test.iloc[:,1]

source_test.to_csv (os.path.join(os.pardir,"/data/02-preprocessed/bicleaner_cleaned/bicleaner_source_test.txt") , index = None, header = False)
target_test.to_csv (os.path.join(os.pardir,"/data/02-preprocessed/bicleaner_cleaned/bicleaner_target_test.txt") , index = None, header = False)

source_dev=dev.iloc[:,0]
target_dev=dev.iloc[:,1]

source_dev.to_csv (os.path.join(os.pardir,"/data/02-preprocessed/bicleaner_cleaned/bicleaner_source_dev.txt") , index = None, header = False)
target_dev.to_csv (os.path.join(os.pardir,"/data/02-preprocessed/bicleaner_cleaned/bicleaner_target_dev.txt") , index = None, header = False)

In [39]:
#write basic_cleaning df's to files size 783895
source_train_basic=basic_cleaning_train.iloc[:,0]
target_train_basic=basic_cleaning_train.iloc[:,1]

source_train_basic.to_csv (os.path.join(os.pardir,"/data/02-preprocessed/basic_cleaned/basic_source_train.txt") , index = None, header = False)
target_train_basic.to_csv (os.path.join(os.pardir,"/data/02-preprocessed/basic_cleaned/basic_target_train.txt") , index = None, header = False)

In [58]:
#write less rules df's to files size 610480
source_train_less_rules=train_less_nodev.iloc[:,0]
target_train_less_rules=train_less_nodev.iloc[:,1]

source_train_less_rules.to_csv ("/home/bernadeta/BA_code/data/02-preprocessed/less_rules/less_source_train.txt" , index = None, header = False)
target_train_less_rules.to_csv ("/home/bernadeta/BA_code/data/02-preprocessed/less_rules/less_target_train.txt" , index = None, header = False)

In [59]:
!wc -l /home/bernadeta/BA_code/data/02-preprocessed/less_rules/*

   610480 /home/bernadeta/BA_code/data/02-preprocessed/less_rules/less_source_train.txt
   610480 /home/bernadeta/BA_code/data/02-preprocessed/less_rules/less_target_train.txt
  1220960 insgesamt
